In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import numpy as np

# 擬似入力データ

In [7]:
# batch_size = 2
user = np.zeros((2,1000))
user[0][1] = 1.0
user[1][444] = 1.0
current_youbi = np.array([[[1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0]],
                          [[1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0]]]).astype("float32")
history_youbi = np.array([[[1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0]],
                          [[1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0], [1,0,0,0,0,0,0]]]).astype("float32")
current = np.random.randint(2, size=(2, 7, 32)).astype("float32")
history = np.random.randint(2, size=(2, 14, 32)).astype("float32")

print(f"user.shape={user.shape}, current_youbi.shape={current_youbi.shape}, history_youti.shape={history_youbi.shape}, current.shape={current.shape}, history.shape={history.shape}")

user.shape=(2, 1000), current_youbi.shape=(2, 7, 7), history_youti.shape=(2, 14, 7), current.shape=(2, 7, 32), history.shape=(2, 14, 32)


# MultiModalEmbedding

In [18]:
class MultiModalEmbedding(Layer):
    def __init__(self):
        super(MultiModalEmbedding, self).__init__()
        self.time_embedding = Dense(16)
        self.user_embedding = Dense(16)
        self.location_embedding = Dense(256)
        
    def call(self, u, current_youbi, history_youbi, current, history):
        user_embed = tf.reshape(self.user_embedding(u), [-1, 1, 16]) #(batch, 1, 16)
        current_embed = self.location_embedding(current) #(batch, 7, 256)
        history_embed = self.location_embedding(history) #(batch, 14, 256)　
        current_time_embed = self.time_embedding(current_youbi) #(batch, 7, 16)
        history_time_embed = self.time_embedding(history_youbi) #(batch, 14, 16)
        return user_embed, current_embed, history_embed, current_time_embed, history_time_embed

## MultiModalEmbedding　テスト

In [19]:
embedding = MultiModalEmbedding()
user_embed, current_embed, history_embed, current_time_embed, history_time_embed = embedding(user, current_youbi, history_youbi, current, history)
print(f"user_embed.shape={user_embed.shape}, current_embed.shape={current_embed.shape}, history_embed.shape={history_embed.shape}, current_time_embed.shape={current_time_embed.shape}, history_time_embed.shape={history_time_embed.shape}")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

user_embed.shape=(2, 1, 16), current_embed.shape=(2, 7, 256), history_embed.shape=(2, 14, 256), current_time_embed.shape=(2, 7, 16), history_time_embed.shape=(2, 14, 16)


# HistoricalAttention

In [20]:
class HistoricalAttention(Layer):
    def __init__(self):
        super(HistoricalAttention, self).__init__()
        self.dense = Dense(256, activation="tanh")
        self.gru = GRU(256, return_sequences=True)
        self.W = Dense(256, use_bias=False)
        
    def call(self, history_embed, time_embed, query):
        history = tf.concat([history_embed, time_embed], axis=2) #(batch, 14, 256+16)
        history = self.dense(history) #(batch, 14, 256)
        candidate = self.gru(history) #(batch, 14, 256)
        
        left = self.W(query) #(batch, 1, 256)
        score = tf.nn.tanh(tf.matmul(left, candidate, transpose_b=True)) #(batch, 1, 14)
        attention_weights = tf.nn.softmax(score) #(batch, 1, 14)
        
        context = tf.matmul(attention_weights, candidate) #(batch, 1, 256)
        
        return context
        

## HistoricalAttention　テスト

In [21]:
attention = HistoricalAttention()
query = np.random.random(size=(2, 1, 256))
context = attention(history_embed, history_time_embed, query)
print(f"context.shape={context.shape}")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

context.shape=(2, 1, 256)


# DeepMove

In [22]:
class DeepMove(Model):
    def __init__(self):
        super(DeepMove, self).__init__()
        self.multi_modal_embedding = MultiModalEmbedding()
        self.historical_attention = HistoricalAttention()
        self.gru = GRU(256)
        self.dense = Dense(32, activation="sigmoid")
        
    def call(self, user, current_youbi, history_youbi, current, history):
        user_embed, current_embed, history_embed, current_time_embed, history_time_embed = self.multi_modal_embedding(user, current_youbi, history_youbi, current, history)
        current = tf.concat([current_embed, current_time_embed], axis=2) #(batch, 7, 256+16)
        query = tf.reshape(self.gru(current), [-1, 1, 256]) #(batch, 1, 256)
        context = self.historical_attention(history_embed, history_time_embed, query) #(batch, 1, 256)
        all_features = tf.concat([context, query, user_embed], axis=2) #(batch, 1, 256+256+16)
        return self.dense(all_features) #(batch, 1, 32)

## DeepMove　テスト

In [23]:
deepmove = DeepMove()
out = deepmove(user, current_youbi, history_youbi, current, history)
print(f"out={out.shape}")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

out=(2, 1, 32)
